## Résumé - outils et résultats

Ici on cherche des récommendations sur le log des chansons écoutées, mais suite à un remaniement qui attribue une 
note (entre 0 et 5) à chaque chanson pour chaque utilisateur. (Remaniement effectué par https://github.com/Patouche). Ce remaniement semble avoir rassemblé les 'doublons' (doublon: un utilisateur écoute chanson x un jour, le réécoute le lendemain) pour que chaque paire utilisateur--chanson paraisse uniquement maintenant une fois, avec son score.

OUTILS: library Surprise

RESULTATS

1. KNNwithMeans - plus des utilisateurs se rassemblent dans leur consomation, plus ils devraient s'échanger de chansons.
On arrive à prédire un score pour une chanson, dans le context d'un utilisateur, et sur (je crois) une échelle de 0 à 5, genre (pour une chanson très appréciée, et une autre peu appréciée):
user 0 & chanson 55915: 4.52 est 
user 0 & chanson 78257: 0.37 est
- mais comme ces données viennent comme un élément à l'intérieur d'un object Surprise, je ne vois pas encore 
comment exploiter cet outil. Il faudrait d'abord isoler le score 'est' d'une longue chaîne de résultats, et ensuite...mettre un boucle à tourner pour prédire chaque score pour un utilisateur? Très peu efficace.

2. Une fois les ensembles 'train' et 'test' établis à partir de notre log, on peut comparer des différents algos
pris de Surprise. On a simplement pris KNNwithMeans mais apparemment c'est BaselineOnly qui a le plus petit RMSE


## importer et remanier

In [36]:

import csv
import pandas as pd

from surprise import KNNBasic
from surprise import NormalPredictor
from surprise import BaselineOnly
from surprise import KNNWithMeans
from surprise import KNNWithZScore
from surprise import KNNBaseline

from surprise import Dataset
from surprise import Reader

from surprise import accuracy
from surprise.model_selection import cross_validate
from surprise.model_selection import train_test_split


In [37]:
# import the log with its 'scores' derived by patouche
log = pd.read_csv('.CSV FILE LOCATION')

# rename the 'count' column because 'count' is a method in python libraries
log.rename(columns = {'song_id':'song','count':'count_score'}, inplace = True)

print(log.shape)
log.head(5)

(169746, 3)


,song,count_score,user_id
0,ghASliN5bAI,5,0
1,y_goHl-GuNk,4,0
2,AuFiBjNTB9o,4,0
3,tSv04ylc6To,3,0
4,XdBlbR3z1jE,2,0


In [38]:
# changer le type de la colonne 'song' à 'catégorie'
log['song'] = log['song'].astype("category")

# créer une colonne 'song ID' qui sera plus facile à manipuler que les strings youtube qui font 'nom'
log['song_id'] = log['song'].cat.codes

# changer le type de song_id aussi pour que la colonne soit 'catégorie'
log.song_id = log.song_id.astype("category")


In [39]:
log.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 169746 entries, 0 to 169745
Data columns (total 4 columns):
song           169746 non-null category
count_score    169746 non-null int64
user_id        169746 non-null int64
song_id        169746 non-null category
dtypes: category(2), int64(2)
memory usage: 10.1 MB


In [40]:
# remanier l'ordre des colonnes pour mieux suivre la logique qui sera appliquée plus bas

log = log[['user_id', 'song_id', 'count_score']]

In [41]:
log.head(3)

,user_id,song_id,count_score
0,0,55915,5
1,0,78600,4
2,0,14553,4


## 1. Kerpanic KNNwithMeans (from surprise) - explicit User User collaborative filtering

https://kerpanic.wordpress.com/2018/03/26/a-gentle-guide-to-recommender-systems-with-surprise/

    USER:USER
    The output is the prediction of user u’s rating on item i:
    si on donnait chanson X à utilisateur, que dirait elle?
    We utilize the similarity measure between user u and user v in this case.
    
    ITEM:ITEM - DO NOT RUN, TAKES FOREVER / HANGS. I DONT KNOW WHY
    Instead of using user similarity, we use item similarity measure to calculate the prediction.
    Ssimilarity is now between item i and item j, instead of user u and v as before.


In [42]:

# on crée / définit un lecteur en précisant l'échelle de notes
reader = Reader(rating_scale = (0, 5))

# on définit 'data' qui prendra comme paramètres les colonnes utilisateur, chanson et score (et le lecteur)
data = Dataset.load_from_df(log[['user_id', 'song_id', 'count_score']], reader=reader)

# on divise le data pour garder 15% pour la partie test
trainset, testset = train_test_split(data, test_size=.15)

# Use user_based true/false to switch between user-based or item-based collaborative filtering
algo = KNNWithMeans(k=50, sim_options={'name': 'pearson_baseline', 'user_based': True})

# on utilise la partie 'training' pour former l'algo sur notre log
algo.fit(trainset)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [43]:
# avec l'algo 'formé' sur notre log, on peut demander des prédictions en précisant des utilisateurs et 
# chansons et en utilisant algo.predict

uid = 1273  # utilisateur 102
iid = 73445  # chanson 2

# get a prediction for specific users and items.

pred = algo.predict(uid, iid, r_ui=4, verbose = True)
pred

# résultat: estimé 0.57

user: 1273       item: 73445      r_ui = 4.00   est = 5.00   {'actual_k': 1, 'was_impossible': False}


Prediction(uid=1273, iid=73445, r_ui=4, est=5, details={'actual_k': 1, 'was_impossible': False})

notons les scores rendu par l'algo quand on demande des chansons aimé / mal aimé par les utilisateurs

user 0 / chanson 55915 (score 5) gives 4.52 est
user 0 / chanson 78257 (score 1) gives 0.37 est
user 1270 / chanson 18125 (score 5) gives 2 est
user 1273 / chanson 73445 (score 5) gives 5 est
user 1274 / chanson 61382 (score 0) gives 0.43 est
user 1274 / chanson 60518 (score 5) gives 5 est


## 2. choisir son algo - d'abord notre KNNwithMeans

In [30]:

# on crée / définit un lecteur en précisant l'échelle de notes
reader = Reader(rating_scale = (0, 5))

# on définit 'data' qui prendra comme paramètres les colonnes utilisateur, chanson et score (et le lecteur)
data = Dataset.load_from_df(log[['user_id', 'song_id', 'count_score']], reader=reader)

# on effectue une 'cross validation' avec ce 'data' et l'algo KNN basic
cross_validate(KNNWithMeans(), data, verbose=False)

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.


{'test_rmse': array([0.85969049, 0.86859054, 0.87027891, 0.84736084, 0.86931079]),
 'test_mae': array([0.52082204, 0.52189573, 0.52227234, 0.5124031 , 0.52228699]),
 'fit_time': (0.2511889934539795,
  0.25397682189941406,
  0.2724578380584717,
  0.24985003471374512,
  0.26448702812194824),
 'test_time': (0.9160909652709961,
  0.7501192092895508,
  0.7518713474273682,
  0.7315359115600586,
  0.898292064666748)}

## ensuite les autre...

In [31]:
# KNN BASIC

cross_validate(KNNBasic(), data, verbose=False)


Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.


{'test_rmse': array([1.02156   , 1.02214966, 1.02343231, 1.03110671, 1.0293117 ]),
 'test_mae': array([0.61771391, 0.6191751 , 0.61788323, 0.61980289, 0.62024611]),
 'fit_time': (0.12078094482421875,
  0.18481993675231934,
  0.19462823867797852,
  0.19391202926635742,
  0.17820382118225098),
 'test_time': (0.9191980361938477,
  0.7210559844970703,
  0.8951308727264404,
  0.7001557350158691,
  0.8903698921203613)}

In [32]:
# KNN BASELINE

cross_validate(KNNBaseline(), data, verbose=False)

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.


{'test_rmse': array([0.83140746, 0.84168616, 0.83390399, 0.83705431, 0.83373868]),
 'test_mae': array([0.44893403, 0.45339807, 0.4463593 , 0.45041308, 0.44958305]),
 'fit_time': (1.0558180809020996,
  1.0849699974060059,
  1.0774669647216797,
  1.065384864807129,
  1.099684238433838),
 'test_time': (1.0611610412597656,
  0.8946239948272705,
  0.9698550701141357,
  0.872157096862793,
  0.9927947521209717)}

In [35]:
# KNN WITH Z SCORE

cross_validate(KNNWithZScore(), data, verbose=False)

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.


{'test_rmse': array([0.87371063, 0.86050918, 0.86127576, 0.85830653, 0.85687639]),
 'test_mae': array([0.50440189, 0.50064981, 0.49920473, 0.49995817, 0.4971901 ]),
 'fit_time': (0.3889899253845215,
  0.4075331687927246,
  0.390394926071167,
  0.41718196868896484,
  0.3983891010284424),
 'test_time': (1.0694599151611328,
  0.8497762680053711,
  0.8397817611694336,
  0.824321985244751,
  0.9847052097320557)}

In [33]:
# NORMAL PREDICTOR

cross_validate(NormalPredictor(), data, verbose=False)

{'test_rmse': array([1.15950194, 1.15777458, 1.15118058, 1.14544566, 1.16207901]),
 'test_mae': array([0.78071995, 0.77742258, 0.77529715, 0.77492037, 0.78557626]),
 'fit_time': (0.30531978607177734,
  0.3954896926879883,
  0.34116315841674805,
  0.35918211936950684,
  0.39243102073669434),
 'test_time': (0.3878488540649414,
  0.5178060531616211,
  0.34609103202819824,
  0.5519559383392334,
  0.36118102073669434)}

In [34]:
# BASELINE ONLY

cross_validate(BaselineOnly(), data, verbose=False)

Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...


{'test_rmse': array([0.76798493, 0.75895732, 0.77700649, 0.76376956, 0.75113828]),
 'test_mae': array([0.40365573, 0.39997534, 0.40860107, 0.40416294, 0.39798417]),
 'fit_time': (0.9371058940887451,
  0.978165864944458,
  0.9650669097900391,
  0.9581189155578613,
  0.9747271537780762),
 'test_time': (0.46396613121032715,
  0.3619987964630127,
  0.43592309951782227,
  0.44666600227355957,
  0.32114672660827637)}